In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, accuracy, run_experiment
from models import GCN_3, ite_GCN

In [2]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="../data/cora/", dataset="cora")

Loading cora dataset...


In [8]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
num_epochs = 200
smooth_fac = 0.2

In [6]:
model1 = GCN_3(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)


Intialize a 3-layer GCN


In [9]:
model2 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=dropout,
            train_nite= 3,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=smooth_fac)

Unspecified or invalid number of iterations for inference
Initialize a 1-layer GCN with  3 iterations
Gradient flows to all iterations:  True


In [7]:
run_experiment(num_epochs=num_epochs, model=model1, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


runrunrun!
Epoch: 0001 loss_train: 2.0306 acc_train: 0.2000 time: 0.0235s
Epoch: 0002 loss_train: 1.9886 acc_train: 0.2000 time: 0.0151s
Epoch: 0003 loss_train: 1.9655 acc_train: 0.2000 time: 0.0185s
Epoch: 0004 loss_train: 1.9333 acc_train: 0.2000 time: 0.0090s
Epoch: 0005 loss_train: 1.9009 acc_train: 0.2000 time: 0.0100s
Epoch: 0006 loss_train: 1.8844 acc_train: 0.2000 time: 0.0100s
Epoch: 0007 loss_train: 1.8505 acc_train: 0.2714 time: 0.0142s
Epoch: 0008 loss_train: 1.8535 acc_train: 0.3357 time: 0.0120s
Epoch: 0009 loss_train: 1.8327 acc_train: 0.3143 time: 0.0108s
Epoch: 0010 loss_train: 1.8131 acc_train: 0.2929 time: 0.0087s
Epoch: 0011 loss_train: 1.7981 acc_train: 0.3000 time: 0.0117s
Epoch: 0012 loss_train: 1.8260 acc_train: 0.2929 time: 0.0082s
Epoch: 0013 loss_train: 1.7958 acc_train: 0.2929 time: 0.0077s
Epoch: 0014 loss_train: 1.7843 acc_train: 0.2929 time: 0.0093s
Epoch: 0015 loss_train: 1.8240 acc_train: 0.2929 time: 0.0083s
Epoch: 0016 loss_train: 1.7817 acc_train: 0.

In [10]:
run_experiment(num_epochs=num_epochs, model=model2, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


runrunrun!
Epoch: 0001 loss_train: 1.9485 acc_train: 0.0929 time: 0.9472s
Epoch: 0002 loss_train: 2.0067 acc_train: 0.2929 time: 0.7092s
Epoch: 0003 loss_train: 1.9422 acc_train: 0.1500 time: 0.6798s
Epoch: 0004 loss_train: 1.9350 acc_train: 0.1500 time: 0.6416s
Epoch: 0005 loss_train: 1.9311 acc_train: 0.1786 time: 0.6771s
Epoch: 0006 loss_train: 1.9162 acc_train: 0.2929 time: 0.6072s
Epoch: 0007 loss_train: 1.8808 acc_train: 0.2357 time: 0.6201s
Epoch: 0008 loss_train: 1.8166 acc_train: 0.2571 time: 0.7154s
Epoch: 0009 loss_train: 1.8278 acc_train: 0.2929 time: 0.5828s
Epoch: 0010 loss_train: 1.7718 acc_train: 0.2929 time: 0.6602s
Epoch: 0011 loss_train: 1.7429 acc_train: 0.2929 time: 0.6073s
Epoch: 0012 loss_train: 1.7089 acc_train: 0.2929 time: 0.6709s
Epoch: 0013 loss_train: 1.6465 acc_train: 0.2929 time: 0.5674s
Epoch: 0014 loss_train: 1.5402 acc_train: 0.2929 time: 0.6092s
Epoch: 0015 loss_train: 1.4454 acc_train: 0.3214 time: 0.6600s
Epoch: 0016 loss_train: 1.3754 acc_train: 0.

In [ ]:
model3 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=dropout,
            nite = 3,
            allow_grad=False,
            smooth_fac=0.3)

In [ ]:
run_experiment(num_epochs=num_epochs, model=model3, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)